In [11]:
import pandas as pd
import os
from datetime import datetime

In [12]:
def load_city_data(city, data_type, source_folder):
    '''Завантаження даних про оренду або продаж житла для заданого міста'''
    city_folder = os.path.join(source_folder, city)
    if data_type == 'sell':
        file_pattern = f'{city}_sell_raw_*.csv'
    elif data_type == 'rent':
        file_pattern = f'{city}_rent_raw_*.csv'
    else:
        raise ValueError("data_type має бути 'sell' або 'rent'")
    
    files = [file for file in os.listdir(city_folder) if file.startswith(file_pattern.split('_raw_')[0])]
    if not files:
        raise FileNotFoundError(f"Не знайдено файлів для міста {city} та типу даних {data_type}")
    
    latest_file = max(files, key=lambda x: os.path.getmtime(os.path.join(city_folder, x)))
    return pd.read_csv(os.path.join(city_folder, latest_file))


In [13]:
def remove_missing_prices(df):
    '''Видалення записів з відсутніми цінами'''
    return df.dropna(subset=['price'])

In [14]:
def fill_missing_features(df):
    '''Заповнення пропущених значень у стовпцях hasParkingSpace, hasBalcony, hasElevator, hasSecurity, hasStorageRoom'''
    features = ['hasParkingSpace', 'hasBalcony', 'hasElevator', 'hasSecurity', 'hasStorageRoom']
    for feature in features:
        if feature in df.columns:
            df[feature] = df[feature].fillna('no')
    return df

In [15]:
def replace_yes_no(df):
    '''Заміна значень "yes" на 1 та "no" на 0'''
    return df.replace({'yes': 1, 'no': 0})

In [16]:
def fill_missing_condition(df):
    '''Заповнення пропущених значень у стовпці condition'''
    if 'condition' in df.columns:
        premium_min = df[df['condition'] == 'premium']['price'].min()
        premium_max = df[df['condition'] == 'premium']['price'].max()
        low_min = df[df['condition'] == 'low']['price'].min()
        low_max = df[df['condition'] == 'low']['price'].max()
        
        def fill_condition(row):
            if pd.isna(row['condition']):
                if premium_min <= row['price'] <= premium_max:
                    return 'premium'
                elif low_min <= row['price'] <= low_max:
                    return 'low'
                else:
                    return 'medium'
            return row['condition']
        
        df['condition'] = df.apply(fill_condition, axis=1)
    return df

In [17]:
def save_clean_data(df, city, data_type, clean_folder):
    # Перевірка наявності папки для міста, якщо її немає - створити
    city_folder = os.path.join(clean_folder, city)
    os.makedirs(city_folder, exist_ok=True)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    file_name = f'clean_data_{data_type}_{city}_{timestamp}.csv'
    file_path = os.path.join(city_folder, file_name)
    df.to_csv(file_path, index=False)

In [18]:
# Основна функція для очищення та нормалізації даних
def clean_and_normalize_data(df):
    df = remove_missing_prices(df)
    df = fill_missing_features(df)
    df = replace_yes_no(df)
    df = fill_missing_condition(df)
    return df


In [19]:
def main():
  source_folder = r'C:\Users\KrisMur\Desktop\PetProjekt1\Apartment_Prices_in_KR-PL-\raw_city_data'
  city = 'Krakow'
  data_type = 'sell'
  clean_folder = r'C:\Users\KrisMur\Desktop\PetProjekt1\Apartment_Prices_in_KR-PL-\clean_city_data'

  # Завантаження даних
  df = load_city_data(city, data_type, source_folder)
    
  # Очищення та нормалізація даних
  df_cleaned = clean_and_normalize_data(df)
    
  # Збереження очищених даних
  save_clean_data(df_cleaned, city, data_type, clean_folder)

In [ ]:
if __name__ == "__main__":
  main()